<a href="https://colab.research.google.com/github/KczBen/tol403-lokaverkefni/blob/main/Lokaverkefni.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone repo data into environment
!git clone https://github.com/KczBen/tol403-lokaverkefni.git

In [ ]:
import pandas as pd

nodes = pd.read_csv('tol403-lokaverkefni/data/nodes.tsv', sep='\t')
edges = pd.read_csv('tol403-lokaverkefni/data/edges.tsv', sep='\t')

# Did we actually get the data?
print(nodes.head(10))
print(edges.head(10))